# Population dynamics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp


def integrate(model, y0, args, t_end, n_points=100):
    sol = solve_ivp(
        model,
        args=args,
        y0=list(y0.values()),
        t_span=(0, t_end),
        t_eval=np.linspace(0, t_end, n_points),
        method="LSODA",
    )
    return pd.DataFrame(sol.y.T, index=sol.t, columns=y0.keys())

## SIR model

- Susceptible (S), Infected (I), Recovered (R)
- Total population (N) = S + I + R

$$\begin{align*}
    \frac{dS}{dt} &= -\beta \frac{S \cdot I}{N}  \\
    \frac{dI}{dt} &= \beta \frac{S \cdot I}{N} - \gamma \cdot I \\
    \frac{dR}{dt} &= \gamma \cdot I \\
\end{align*}$$

Let's start with a literal translation of that description.

In [ ]:
def sir_v1(t, y, beta, gamma):
    s, i, r = y
    n = s + i + r

    dsdt = -beta * s * i / n
    didt = beta * s * i / n - gamma * i
    drdt = gamma * i
    return dsdt, didt, drdt


df = integrate(
    sir_v1,
    y0={"S": 0.9, "I": 0.1, "R": 0},
    args=(2, 0.1),
    t_end=10,
)

ax = df.plot()

There is a lot of duplication in the code above (e.g. `beta * s * i / n` appears twice), let's improve on that.

In [ ]:
def sir_v2(t, y, beta, gamma):
    s, i, r = y
    n = s + i + r

    infection = beta * s * i / n
    recovery = gamma * i

    dsdt = -infection
    didt = infection - recovery
    drdt = recovery
    return dsdt, didt, drdt


df = integrate(
    sir_v2,
    y0={"S": 0.9, "I": 0.1, "R": 0},
    args=(2, 0.1),
    t_end=10,
)

ax = df.plot()